In [365]:
import pandas as pd
import numpy as np
from repo_funciones import * #repositorio propio de funciones
import re #para expresiones regulares

In [366]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data = pd.read_csv('../Sharknado/attacks_messy.csv', encoding='utf-8',encoding_errors="ignore" )

Exploración Inicial

In [368]:
dim_original = data.shape #Registramos dimensiones iniciales del archivo para luego ir comparando
dim_original

(25723, 24)

In [369]:
#data.head() Usamos estas funciones para ir haciendonos una idea de los datos y las columnas
#data.tail() Con este nos damos cuenta de que al final del archivo hay filas enteras con valores nulos

In [370]:
#data.info() #Estimación de que columnas necesitan mas trabajo. Por gusto mío prefiero verlo a la inversa -> que columnas tienen mas valores nulos.

In [371]:
nancols(data) #Conteo de valores nulos por col, ascending False.

Unnamed: 22               25722
Unnamed: 23               25721
Time                      22775
Species                   22259
Age                       22252
Sex                       19986
Activity                  19965
Location                  19961
Fatal (Y/N)               19960
Area                      19876
Name                      19631
Country                   19471
Injury                    19449
Investigator or Source    19438
Type                      19425
Year                      19423
href formula              19422
Date                      19421
pdf                       19421
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Case Number               17021
dtype: int64

In [372]:
nancolsper(data) #conteo de valores nulos como % del total de la columna. Nos da una mejor estimación 

Unnamed: 22               100.00
Unnamed: 23                99.99
Time                       88.54
Species                    86.53
Age                        86.51
Sex                        77.70
Activity                   77.62
Location                   77.60
Fatal (Y/N)                77.60
Area                       77.27
Name                       76.32
Country                    75.69
Injury                     75.61
Investigator or Source     75.57
Type                       75.52
Year                       75.51
href formula               75.50
Date                       75.50
pdf                        75.50
href                       75.50
Case Number.1              75.50
Case Number.2              75.50
original order             75.47
Case Number                66.17
dtype: float64

Limpieza duplicados y columnas vacias

In [373]:
data= data.drop_duplicates()

In [374]:
data.duplicated().unique()

array([False])

In [375]:
data=data.drop(nanrows(data)[nanrows(data)> 19]) #Si en una fila 90% de los valores son nulos, se descartan

In [376]:
data = data.reset_index(drop=True)

In [377]:
data = data.drop(nanrows(data)[nanrows(data) >= 15]) #Si en una fila 60% de los valores son nulos, se descartan.

In [378]:
dim_original, data.shape

((25723, 24), (6306, 24))

In [379]:
data = data.reset_index(drop=True)

In [380]:
nancolsper(data) #Unamed 22 y 23 estan vacías y no aportan nada. Se descartan.

Unnamed: 22               99.98
Unnamed: 23               99.97
Time                      53.30
Species                   45.15
Age                       45.04
Sex                        9.12
Activity                   8.79
Location                   8.72
Fatal (Y/N)                8.69
Area                       7.37
Name                       3.49
Country                    0.95
Injury                     0.60
Investigator or Source     0.43
Type                       0.22
Year                       0.19
href formula               0.17
Date                       0.16
pdf                        0.16
href                       0.16
Case Number.1              0.16
Case Number.2              0.16
original order             0.05
Case Number                0.03
dtype: float64

In [381]:
data.drop(["Unnamed: 22", "Unnamed: 23"], axis=1, inplace=True)  #Reminder, axis 0 son filas y axis columnas

In [382]:
nancolsper(data) #Time,species y age, la proporción de valores nulos es demasiado representativa para poder rellenar con media/moda ya que introduciría un sesgo muy importante.

Time                      53.30
Species                   45.15
Age                       45.04
Sex                        9.12
Activity                   8.79
Location                   8.72
Fatal (Y/N)                8.69
Area                       7.37
Name                       3.49
Country                    0.95
Injury                     0.60
Investigator or Source     0.43
Type                       0.22
Year                       0.19
href formula               0.17
Date                       0.16
Case Number.2              0.16
pdf                        0.16
href                       0.16
Case Number.1              0.16
original order             0.05
Case Number                0.03
dtype: float64

In [383]:
data.columns = data.columns.str.strip().str.capitalize().str.title().str.replace(':','').str.replace(' ','_').str.replace('.','_')

Limpieza Columna "Time"

In [384]:
#data["Time"].unique() #Vemos hay muchas inconsistencias en los valores, hay que homogenizar

In [385]:
#unique_cols(data) #364 valores unicos 
#get_unique_dtypes(data, "Time") #con strings y con floats. Se intuye floats seran los valores nulos.

In [386]:
data["Time"]=data["Time"].fillna("unknown")

In [387]:
data["Time"] = data["Time"].str.replace('h',':')

In [388]:
#convención de tiempo
#Morning: 6am a 12
#afternoon: 12 a 7
#night: 8 a 5am
#Asumimos que todos ingresaron la hora en formato 24h
data.loc[data["Time"]=="14:00  -15:00", "Time"]="14:30"
data.loc[data["Time"]=="Late afternoon", "Time"]="18:00"
data.loc[data["Time"]=="Morning", "Time"]="09:00"
data.loc[data["Time"]=="19:00, Dusk", "Time"]="19:00"
data.loc[data["Time"]=="Nig:t", "Time"]="21:00"
data.loc[data["Time"]=="Afternoon", "Time"]="17:00"
data.loc[data["Time"]=="Midday", "Time"]="13:00"
data.loc[data["Time"]=="S:ortly before 12:00", "Time"]="11:30"
data.loc[data["Time"]=="After noon", "Time"]="13:00"
data.loc[data["Time"]=="14:30 / 15:30", "Time"]="15:00"
data.loc[data["Time"]=="Midnig:t", "Time"]="24:00"
data.loc[data["Time"]=="09:30 / 10:00", "Time"]="09:40"
data.loc[data["Time"]=="10:45-11:15", "Time"]="11:00"
data.loc[data["Time"]=="Evening", "Time"]="20:00"
data.loc[data["Time"]=="Sometime between 06:00 & 08:oo", "Time"]="07:00"
data.loc[data["Time"]=="Early afternoon", "Time"]="14:00"
data.loc[data["Time"]=="18:15-18:30", "Time"]="18:20"
data.loc[data["Time"]=="09:00 - 09:30", "Time"]="09:15"
data.loc[data["Time"]=="0830", "Time"]="08:30"
data.loc[data["Time"]=="Just before noon", "Time"]="11:30"
data.loc[data["Time"]=="1600", "Time"]="16:00"
data.loc[data["Time"]=="Early morning", "Time"]="09:00"
data.loc[data["Time"]=="Dawn", "Time"]="07:00"
data.loc[data["Time"]=="AM", "Time"]="07:00"
data.loc[data["Time"]=="A.M", "Time"]="07:00"
data.loc[data["Time"]=="Dusk", "Time"]="20:00"
data.loc[data["Time"]=="Lunc:time", "Time"]="13:00"
data.loc[data["Time"]=="15j45", "Time"]="15:45"
data.loc[data["Time"]=="0500", "Time"]="05:00"
data.loc[data["Time"]=="Before 07:00", "Time"]="06:00"
data.loc[data["Time"]=="10:00 -- 11:00", "Time"]="10:30"
data.loc[data["Time"]=='"Just before 11:00"', "Time"]="10:30"
data.loc[data["Time"]=="Sunset", "Time"]="19:30"
data.loc[data["Time"]=="Just before sundown", "Time"]="19:30"
data.loc[data["Time"]=="Between 05:00 and 08:00", "Time"]="06:30"
data.loc[data["Time"]=="17:00 or 17:40", "Time"]="17:20"
data.loc[data["Time"]==">08:00", "Time"]="08:20"
data.loc[data["Time"]=="--", "Time"]="unknown"
data.loc[data["Time"]=="Just after 12:00", "Time"]="12:30"
data.loc[data["Time"]=="Early Morning", "Time"]="08:30"
data.loc[data["Time"]=="S:ortly after midnig:t", "Time"]="24:30"
data.loc[data["Time"]=="09:00 -10:00", "Time"]="09:30"
data.loc[data["Time"]=="20:45 (Sunset)", "Time"]="20:45"
data.loc[data["Time"]=="Late morning", "Time"]="11:00"
data.loc[data["Time"]=="P.M.", "Time"]="14:00"
data.loc[data["Time"]=="S:ortly before 13:00", "Time"]="12:00"
data.loc[data["Time"]=="8:04 pm", "Time"]="08:00"
data.loc[data["Time"]=="Possibly same incident as 2000.08.21", "Time"]="unknown"
data.loc[data["Time"]=="After Dusk", "Time"]="09:00"
data.loc[data["Time"]=="Noon", "Time"]="12:00"
data.loc[data["Time"]=="2 :ours after Opperman", "Time"]="unknown"
data.loc[data["Time"]=="Mid afternoon", "Time"]="14:00"
data.loc[data["Time"]=="Mid morning", "Time"]="11:00"
data.loc[data["Time"]=="11:00 / 11:30", "Time"]="11:00"
data.loc[data["Time"]=='"Nig:t"', "Time"]="21:00"
data.loc[data["Time"]=='18:30?', "Time"]="18:30"
data.loc[data["Time"]=='30 minutes after 1992.07.08.a', "Time"]="unknown"
data.loc[data["Time"]==">06:45", "Time"]="06:45"
data.loc[data["Time"]=="Between 06:00 & 07:20", "Time"]="06:45"
data.loc[data["Time"]=="<07:30", "Time"]="06:45"
data.loc[data["Time"]=="17:00 Sunset", "Time"]="17:00"
data.loc[data["Time"]=="Nig:tfal", "Time"]="20:00"
data.loc[data["Time"]=="X", "Time"]="unknown"
data.loc[data["Time"]=="18:30 (Sunset)", "Time"]="18:30"
data.loc[data["Time"]=="06j00", "Time"]="06:00"
data.loc[data["Time"]=="Prior to 10:37", "Time"]="10:00"
data.loc[data["Time"]=="Daybreak", "Time"]="20:00"
data.loc[data["Time"]==">12:00", "Time"]="12:00"
data.loc[data["Time"]=="Mid-morning", "Time"]="11:00"
data.loc[data["Time"]=="16:30 or 18:00", "Time"]="18:00"
data.loc[data["Time"]=="Just before dawn", "Time"]="19:00"
data.loc[data["Time"]=="Daytime", "Time"]="10:00"
data.loc[data["Time"]=="Dark", "Time"]="23:00"
data.loc[data["Time"]=="10:00 / 11:00", "Time"]="10:00"
data.loc[data["Time"]=='"After lunc:"', "Time"]="16:00"
data.loc[data["Time"]=='15:00 or 15:45', "Time"]="15:00"
data.loc[data["Time"]=='>17:00', "Time"]="17:00"
data.loc[data["Time"]=='19:00 / 20:00', "Time"]="19:00"
data.loc[data["Time"]=="12:45 / 13:45", "Time"]="13:00"
data.loc[data["Time"]=="14:00 - 15:00", "Time"]="14:00"
data.loc[data["Time"]=="nig:t", "Time"]="20:00"
data.loc[data["Time"]=="03:45 - 04:00", "Time"]="03:00"
data.loc[data["Time"]=="Late nig:t", "Time"]="23:00"
data.loc[data["Time"]=="10:30 or 13:30", "Time"]="12:00"
data.loc[data["Time"]=="15:00j", "Time"]="15:00"
data.loc[data["Time"]=="Midday.", "Time"]="12:00"
data.loc[data["Time"]=='"After dark"', "Time"]="23:00"
data.loc[data["Time"]=='10:00 or 14:00"', "Time"]="23:00"
data.loc[data["Time"]=="2 :rs before sunset", "Time"]="16:00"
data.loc[data["Time"]=="18:15 to 21:30", "Time"]="16:00"
data.loc[data["Time"]=="1500", "Time"]="15:00"
data.loc[data["Time"]=="A.M.", "Time"]="09:00"
data.loc[data["Time"]=="Morning ", "Time"]="09:00"
data.loc[data["Time"]=="07:00 - 08:00", "Time"]="07:00"
data.loc[data["Time"]=="  ", "Time"]="unknown"
data.loc[data["Time"]=='"Evening"', "Time"]="unknown"
data.loc[data["Time"]=="11:30 ", "Time"]="11:30"
data.loc[data["Time"]==' ', "Time"]="unknown"
data.loc[data["Time"]=="Nig:tfall", "Time"]="21:00"
data.loc[data["Time"]=="09:30 / 15:30", "Time"]="12:00"
data.loc[data["Time"]=="12:00 to 14:00", "Time"]="13:00"
data.loc[data["Time"]=="10:00 or 14:00", "Time"]="13:00"
data.loc[data["Time"]=='"s:ortly before dusk"', "Time"]="20:00"
data.loc[data["Time"]==">17:30", "Time"]="18:00"
data.loc[data["Time"]==">14:30", "Time"]="15:00"
data.loc[data["Time"]=="Between 11:00 & 12:00", "Time"]="11:00"
data.loc[data["Time"]=="After 04:00", "Time"]="05:00"
data.loc[data["Time"]=="11:01 -time of s:ip sinking", "Time"]="11:00"
data.loc[data["Time"]=="S:ip aban-doned at 03:10", "Time"]="03:00"
data.loc[data["Time"]=="After dusk", "Time"]="22:00"
data.loc[data["Time"]=="FATAL  (Wire netting installed at local beac:es after t:is incident.)", "Time"]="unknown"
data.loc[data["Time"]=="After midnig:t", "Time"]="01:00"
data.loc[data["Time"]=="Late afternon", "Time"]="18:00"
data.loc[data["Time"]=='"Early evening"', "Time"]="20:00"
data.loc[data["Time"]=="Late Afternoon", "Time"]="18:00"
data.loc[data["Time"]=="   ", "Time"]="unknown"
data.loc[data["Time"]=="Before daybreak", "Time"]="19:00"
data.loc[data["Time"]=="dusk", "Time"]="20:00"
data.loc[data["Time"]=="Before 10:30", "Time"]="20:00"
data.loc[data["Time"]=="06:00 -- 07:00", "Time"]="06:00"
data.loc[data["Time"]=="17:00-18:00", "Time"]="17:00"
data.loc[data["Time"]=="19:00-20:00", "Time"]="19:00"
data.loc[data["Time"]=="1300", "Time"]="13:00"
data.loc[data["Time"]=="11:115", "Time"]="11:00"
data.loc[data["Time"]=="13:345", "Time"]="11:00"
data.loc[data["Time"]=="08:00 / 09:30", "Time"]="08:00"
data.loc[data["Time"]==" 14:00", "Time"]="14:00"

In [389]:
#note to self, hubieses empezado quitando valores especiales/espaciosR.L y te hubieses ahorrado la mitad de este chorizo :,)
#data["Time"].unique()
get_unique_dtypes(data, "Time")

array([<class 'str'>], dtype=object)

In [390]:
data["Time_groups"] = data['Time'].apply(time_of_day)

In [391]:
col_unique_counts(data, "Time_groups")

,Time_groups,Count
0,No information,3378
1,Afternoon,1702
2,Morning,1124
3,Night,102


Limpieza especies 

In [392]:
data["Species"]=data["Species"].fillna("unknown")

Limpieza "Age"

In [393]:
#quitamos espacios y caracteres especiales
data["Age"] = data["Age"].str.replace(r'[^\w\s]|[\&\(\)\.\"&gt;&lt;\?\']', '', regex=True)


In [394]:
data["Age"] = data["Age"].str.rstrip()
data["Age"] = data["Age"].str.lstrip()


In [395]:
data["Age"]=data["Age"].fillna("0")

In [396]:
data.loc[data["Age"]=="Teen", "Age"]="17"
data.loc[data["Age"]=="40s", "Age"]="40"
data.loc[data["Age"]=="60s", "Age"]="60"
data.loc[data["Age"]=="18 monhs", "Age"]="1"
data.loc[data["Age"]=="een", "Age"]="17"
data.loc[data["Age"]=="18 or 20", "Age"]="20"
data.loc[data["Age"]=="30s", "Age"]="30" #mental note: se podria haber hecho algo para quitar toda las "s"
data.loc[data["Age"]=="50s", "Age"]="50"
data.loc[data["Age"]=="28  26", "Age"]="27"
data.loc[data["Age"]=="12 or 13", "Age"]="13"
data.loc[data["Age"]=="46  34", "Age"]="40"
data.loc[data["Age"]=="28 23  30", "Age"]="25"
data.loc[data["Age"]=="36  26", "Age"]="30"
data.loc[data["Age"]=="8 or 10", "Age"]="9"
data.loc[data["Age"]=="30 or 36", "Age"]="32"
data.loc[data["Age"]=="33 or 37", "Age"]="35"
data.loc[data["Age"]=="mid30s", "Age"]="35"
data.loc[data["Age"]=="23  20", "Age"]="21"
data.loc[data["Age"]=="7          31", "Age"]="31"
data.loc[data["Age"]=="32  30", "Age"]="31"
data.loc[data["Age"]=="16 o 18", "Age"]="17"
data.loc[data["Age"]=="Edery", "Age"]="70"
data.loc[data["Age"]=="mid20s", "Age"]="25"
data.loc[data["Age"]=="Ca 33", "Age"]="33"
data.loc[data["Age"]=="21 or 26", "Age"]="23"
data.loc[data["Age"]=="18 o 22", "Age"]="20"
data.loc[data["Age"]=="9  12", "Age"]="12"
data.loc[data["Age"]=="9 monhs", "Age"]="1"
data.loc[data["Age"]=="25 o 35", "Age"]="27"
data.loc[data["Age"]=="23  26", "Age"]="24"
data.loc[data["Age"]=="33  37", "Age"]="35"
data.loc[data["Age"]=="25 or 28", "Age"]="27"
data.loc[data["Age"]=="37 67 35 27    27", "Age"]="40"
data.loc[data["Age"]=="21 3424  35", "Age"]="35"
data.loc[data["Age"]=="30  32", "Age"]="31"
data.loc[data["Age"]=="50  30", "Age"]="40"
data.loc[data["Age"]=="17  35", "Age"]="27"
data.loc[data["Age"]=="X", "Age"]="0"
data.loc[data["Age"]=="middeae", "Age"]="40"
data.loc[data["Age"]=="13 or 18", "Age"]="15"
data.loc[data["Age"]=="34  19", "Age"]="25"
data.loc[data["Age"]=="33  26", "Age"]="28"
data.loc[data["Age"]=="2 o 3 monhs", "Age"]="1"
data.loc[data["Age"]=="MAKE LINE GREEN", "Age"]="0"
data.loc[data["Age"]=="youn", "Age"]="1"
data.loc[data["Age"]=="7 or 8", "Age"]="7"
data.loc[data["Age"]=="17  16", "Age"]="17"
data.loc[data["Age"]=="F", "Age"]="0"
data.loc[data["Age"]=="Boh 11", "Age"]="11"
data.loc[data["Age"]=="9 or 10", "Age"]="9"
data.loc[data["Age"]=="36  23", "Age"]="32"
data.loc[data["Age"]=="AM", "Age"]="0"
data.loc[data["Age"]=="10 or 12", "Age"]="11"
data.loc[data["Age"]=="31 or 33", "Age"]="32"
data.loc[data["Age"]=="13 or 14", "Age"]="13"
data.loc[data["Age"]=="Teens", "Age"]="15"
data.loc[data["Age"]=="", "Age"]="0"
data.loc[data["Age"]=="adu", "Age"]="25"
data.loc[data["Age"]=="20s", "Age"]="25"
#ULTIMA VEZ, la proxima si valor no numerico se considera nulo y ya

In [397]:
data["Age"] = pd.to_numeric(data["Age"], errors='coerce').astype(int) #pasamos de STR a Int para separar por grupos de edad

In [398]:
data["Age"].unique()

array([57, 11, 48,  0, 18, 52, 15, 12, 32, 10, 21, 34, 30, 60, 41, 37, 56,
       19, 25, 69, 38, 55, 35, 46, 45, 14, 40, 28, 20, 54, 24, 26, 49, 22,
        7, 31, 17, 33, 13, 42,  3,  8, 50, 16, 82, 73, 29, 68, 51, 39, 58,
       47, 61, 65, 36, 66, 43,  9, 72, 59,  6, 27, 64, 23, 71, 44, 62, 63,
       70,  1, 53, 77, 74,  5, 86, 84, 75, 87, 67, 81, 78,  2])

In [399]:
data["Age"]=data["Age"].fillna("0")

In [400]:
get_unique_dtypes(data, "Age")

array([<class 'int'>], dtype=object)

In [401]:
data["Age"] = data["Age"].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6306 entries, 0 to 6305
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case_Number             6304 non-null   object 
 1   Date                    6296 non-null   object 
 2   Year                    6294 non-null   float64
 3   Type                    6292 non-null   object 
 4   Country                 6246 non-null   object 
 5   Area                    5841 non-null   object 
 6   Location                5756 non-null   object 
 7   Activity                5752 non-null   object 
 8   Name                    6086 non-null   object 
 9   Sex                     5731 non-null   object 
 10  Age                     6306 non-null   int32  
 11  Injury                  6268 non-null   object 
 12  Fatal_(Y/N)             5758 non-null   object 
 13  Time                    6306 non-null   object 
 14  Species                 6306 non-null   

In [402]:
data["Age_Groups"] = data["Age"].apply(age_groups,"Age")

In [403]:
data.head()

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species,Investigator_Or_Source,Pdf,Href_Formula,Href,Case_Number_1,Case_Number_2,Original_Order,Time_groups,Age_Groups
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18:00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,Afternoon,adult
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14:30,unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,Afternoon,kid
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07:45,unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,Morning,adult
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,0,Minor injury to lower leg,N,unknown,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,No information,No information
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,0,Lacerations to leg & hand shark PROVOKED INCIDENT,N,unknown,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,No information,No information


Limpieza Columna sex 

In [404]:
data.loc[data["Sex"] == "lli", "Sex"]="M" 

In [405]:
data.loc[data["Sex"]== "M ", "Sex"]="M"

In [406]:
data.loc[data["Sex"]=="N", "Sex" ]="M"

In [407]:
data.loc[data["Sex"]==".", "Sex"]="unknown"

In [408]:
data["Sex"]=data["Sex"].fillna("unknown")

In [409]:
data["Sex"].unique()

array(['F', 'M', 'unknown'], dtype=object)

Limpieza Activity

In [410]:
nancolsper(data) #vamos a rellenar por la moda porque no tiene tanto caso, no necesitamos tanto detalle ni los valores nulos son demasiado representativos como para introducir sesgo.

Activity                  8.79
Location                  8.72
Fatal_(Y/N)               8.69
Area                      7.37
Name                      3.49
Country                   0.95
Injury                    0.60
Investigator_Or_Source    0.43
Type                      0.22
Year                      0.19
Href_Formula              0.17
Case_Number_2             0.16
Date                      0.16
Pdf                       0.16
Href                      0.16
Case_Number_1             0.16
Original_Order            0.05
Case_Number               0.03
dtype: float64

In [411]:
#data["Activity"].value_counts()

In [412]:
activitymode = data["Activity"].mode()
activitymode

0    Surfing
Name: Activity, dtype: object

In [413]:
#data["Activity"]=data["Activity"].fillna(activitymode) #esta me funciono pero luego no for some reason
data["Activity"].fillna("Surfing", inplace=True)

In [414]:
nancolsper(data)

Location                  8.72
Fatal_(Y/N)               8.69
Area                      7.37
Name                      3.49
Country                   0.95
Injury                    0.60
Investigator_Or_Source    0.43
Type                      0.22
Year                      0.19
Href_Formula              0.17
Case_Number_2             0.16
Date                      0.16
Pdf                       0.16
Href                      0.16
Case_Number_1             0.16
Original_Order            0.05
Case_Number               0.03
dtype: float64

Limpieza Location

In [415]:
#Aquí no tiene tanto sentido llenar con modas/medias porque este dato se entrelaza con country y area. Mejor llenamos con unknown

In [416]:
data.loc[data[["Country", "Area", "Location"]].isna().all(axis=1), ["Country", "Area", "Location"]] = "unknown"

In [417]:
data.loc[data[["Area", "Location"]].isna().all(axis=1), [ "Area", "Location"]] = "unknown"

In [418]:
#data.loc[data[["Country", "Location"]].isna().all(axis=1), [ "Country", "Location"]] = "unknown"

In [419]:
data["Location"]=data["Location"].fillna("unknown")

In [420]:
#data.loc[(data["Country"].isna()) & (data["Area"].notna())]

In [421]:
data.iloc[2950,4]="England"
data.iloc[3381,4]="Saint Kitts & Nevis"
data.iloc[4260,4]="Madagascar"
data.iloc[4012,4]="Australia"
data.iloc[4225,4]="India"
data.iloc[4492,4]="Costa Rica"
data.iloc[4706,4]="Virgin Islands"
data.iloc[5014,4]="France"
data.iloc[5736,4]="USA"
data.iloc[5802,4]="Australia"
data.iloc[6131,4]="St Helena"
data.iloc[6169,4]="Italy"
data.iloc[6171,4]="Italy"

In [422]:
data.iloc[3599,4]="USA"
data.iloc[4633,4]="Hong Kong"
data.iloc[5804,4]="USA"
data.iloc[6149,4]="Australia"
data.iloc[6200,4]="India"

In [423]:
data["Country"]=data["Country"].fillna("unknown")

In [424]:
data["Country"] = data["Country"].str.replace(r'[^\w\s]|[\&\(\)\.\"&gt;&lt;\?\']', '', regex=True)

In [425]:
data["Country"] = data["Country"].str.strip().str.capitalize().str.title()

In [426]:
data["Area"] = data["Area"].fillna(data["Location"])

In [427]:
nancols(data)

Fatal_(Y/N)               548
Name                      220
Injury                     38
Investigator_Or_Source     27
Type                       14
Year                       12
Href_Formula               11
Date                       10
Pdf                        10
Href                       10
Case_Number_1              10
Case_Number_2              10
Original_Order              3
Case_Number                 2
dtype: int64

Limpieza FATAL(Y/N) 

In [428]:
data["Fatal_(Y/N)"].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [429]:
data.loc[(data["Fatal_(Y/N)"].isna()) & (data["Injury"] == "FATAL"), "Fatal_(Y/N)"] = "Y"

In [430]:
data.loc[data["Fatal_(Y/N)"] == 'M', "Fatal_(Y/N)"] = "N" #injury no parece indicar que murió, ponemos no

In [431]:
data.loc[data["Fatal_(Y/N)"] == "2017", "Fatal_(Y/N)"]="N"

In [432]:
data.loc[data["Fatal_(Y/N)"]=="y", "Fatal_(Y/N)"]="Y"

In [433]:
data["Fatal_(Y/N)"] = data["Fatal_(Y/N)"].str.rstrip()
data["Fatal_(Y/N)"] = data["Fatal_(Y/N)"].str.lstrip()

In [434]:
data["Fatal_(Y/N)"].unique()

array(['N', 'Y', nan, 'UNKNOWN'], dtype=object)

In [435]:
data["Fatal_(Y/N)"]=data["Fatal_(Y/N)"].fillna("UNKNOWN")

In [436]:
nancols(data)

Name                      220
Injury                     38
Investigator_Or_Source     27
Type                       14
Year                       12
Href_Formula               11
Date                       10
Pdf                        10
Href                       10
Case_Number_1              10
Case_Number_2              10
Original_Order              3
Case_Number                 2
dtype: int64

Limpieza columna "Name"/"Investigator_Or_Source"

In [437]:
data["Name"]=data["Name"].fillna("unknown") #no es un dato que nos interese. 

In [438]:
data["Investigator_Or_Source"]=data["Investigator_Or_Source"].fillna("unknown") 

Limpieza Columna "Injury"

In [439]:
data.iloc[6134,11]="FATAL"

In [440]:
data["Injury"]=data["Injury"].fillna("unknown")

In [441]:
nancols(data)

Type              14
Year              12
Href_Formula      11
Date              10
Pdf               10
Href              10
Case_Number_1     10
Case_Number_2     10
Original_Order     3
Case_Number        2
dtype: int64

Limpieza Columna Type 

In [442]:
col_unique_counts(data, "Type")

,Type,Count
0,Unprovoked,4589
1,Provoked,574
2,Invalid,547
3,Sea Disaster,239
4,Boating,203
5,Boat,137
6,Questionable,2
7,Boatomg,1


In [443]:
data["Type"] = data["Type"].replace("Boating", "Boat")
data["Type"] = data["Type"].replace("Boatomg", "Boat")
data["Type"] = data["Type"].replace("Questionable", "Invalid")

In [444]:
data["Type"]=data["Type"].fillna("Invalid") 

In [445]:
col_unique_counts(data, "Type")

,Type,Count
0,Unprovoked,4589
1,Provoked,574
2,Invalid,563
3,Boat,341
4,Sea Disaster,239


Limpieza columna "Case Number"/"Year" y "Href Formula"

In [446]:
data.loc[(data["Case_Number"].isna())]

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species,Investigator_Or_Source,Pdf,Href_Formula,Href,Case_Number_1,Case_Number_2,Original_Order,Time_groups,Age_Groups
5482,NaN,Reported 06-Sep-1905,1905.0,Provoked,Usa,Florida,"Fort Pierce, St Lucie County",Fishing,David Curry,M,0,Lacerations to leg from hooked shark PROVOKED ...,N,unknown,unknown,"Muncie Evening Press, 8/6/1913",1905.09.06.R-Curry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1905.09.06.R,1905.09.06.R,815.0,No information,No information
6304,NaN,NaN,NaN,Invalid,Unknown,unknown,unknown,Surfing,unknown,unknown,0,unknown,UNKNOWN,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,No information,No information


In [447]:
data= data.drop(6304)

In [448]:
data.loc[(data["Case_Number"].isna()) | (data["Case_Number"] == "0")]

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species,Investigator_Or_Source,Pdf,Href_Formula,Href,Case_Number_1,Case_Number_2,Original_Order,Time_groups,Age_Groups
5482,NaN,Reported 06-Sep-1905,1905.0,Provoked,Usa,Florida,"Fort Pierce, St Lucie County",Fishing,David Curry,M,0,Lacerations to leg from hooked shark PROVOKED ...,N,unknown,unknown,"Muncie Evening Press, 8/6/1913",1905.09.06.R-Curry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1905.09.06.R,1905.09.06.R,815.0,No information,No information
6296,0,NaN,NaN,Invalid,Unknown,unknown,unknown,Surfing,unknown,unknown,0,unknown,UNKNOWN,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,6304.0,No information,No information
6297,0,NaN,NaN,Invalid,Unknown,unknown,unknown,Surfing,unknown,unknown,0,unknown,UNKNOWN,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,6305.0,No information,No information
6298,0,NaN,NaN,Invalid,Unknown,unknown,unknown,Surfing,unknown,unknown,0,unknown,UNKNOWN,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,6306.0,No information,No information
6299,0,NaN,NaN,Invalid,Unknown,unknown,unknown,Surfing,unknown,unknown,0,unknown,UNKNOWN,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,6307.0,No information,No information
6300,0,NaN,NaN,Invalid,Unknown,unknown,unknown,Surfing,unknown,unknown,0,unknown,UNKNOWN,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,6308.0,No information,No information
6301,0,NaN,NaN,Invalid,Unknown,unknown,unknown,Surfing,unknown,unknown,0,unknown,UNKNOWN,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,6309.0,No information,No information
6302,0,NaN,NaN,Invalid,Unknown,unknown,unknown,Surfing,unknown,unknown,0,unknown,UNKNOWN,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,6310.0,No information,No information
6303,0,NaN,NaN,Invalid,Unknown,unknown,unknown,Surfing,unknown,unknown,0,unknown,UNKNOWN,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,No information,No information


In [449]:
data = data.drop(data[data["Case_Number"] == "0"].index)

In [450]:
data.iloc[5482,0]="1905.09.06.R"

In [451]:
nancols(data)

Year              3
Href_Formula      2
Date              1
Pdf               1
Href              1
Case_Number_1     1
Case_Number_2     1
Original_Order    1
dtype: int64

In [452]:
data = data.drop(data[data["Case_Number"] == "xx"].index)

In [453]:
data.loc[(data["Href_Formula"].isna())]

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species,Investigator_Or_Source,Pdf,Href_Formula,Href,Case_Number_1,Case_Number_2,Original_Order,Time_groups,Age_Groups
3238,1975.01.19,19-Jan-1975,1975.0,Unprovoked,Australia,South Australia,Coffin Bay,Surfing,David Barrowman,M,17,"FATAL, body not recovered",Y,unknown,unknown,"J. West; Adelaide Advertiser, 1/20/1975; P. Ke...",1975.01.19-Barrowman.pdf,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,1975.01.19,1975.01.19,3059.0,No information,teen


In [454]:
data["Href_Formula"].fillna(data["Href"], inplace=True)

In [455]:
data.loc[(data["Year"].isna())]

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species,Investigator_Or_Source,Pdf,Href_Formula,Href,Case_Number_1,Case_Number_2,Original_Order,Time_groups,Age_Groups
181,2017.01.08.R,Reported 08-Jan-2017,NaN,Invalid,Australia,Queensland,unknown,Spearfishing,Kerry Daniel,M,35,"No attack, shark made a threat display",UNKNOWN,unknown,Bull shark,Liquid Vision 1/8/2017,2017.01.08.R-KerryDaniel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.01.08.R,2017.01.08.R,6116.0,No information,adult
6073,1836.08.19.R,Reported 19-Aug-1836,NaN,Unprovoked,England,Cumberland,Whitehaven,Swimming,a boy,M,0,FATAL,Y,unknown,unknown,"C. Moore, GSAF",1835.08.19.R-Whitehaven.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1836.08.19.R,1836.08.19.R,224.0,No information,No information


In [456]:
data.iloc[181,2]=2017
data.iloc[6073,2]=1836

In [457]:
nancols(data)

Series([], dtype: int64)

In [458]:
data["Year"] = data["Year"].astype(int)

Limpieza Columna "Date"

In [459]:
#col_unique_counts(data, "Date")

In [460]:
data["Date"] = data["Date"].str.replace("Before ", "")
data["Date"] = data["Date"].str.replace("Reported ", "")

In [461]:
data["Date"] = data["Date"].str.rstrip()
data["Date"] = data["Date"].str.lstrip()

In [462]:
#(data["Date"].apply(len) != 11).value_counts() #Por constraints del profe no puedo borrar estas filas con formato fecha incorrecto ya que bajaría por debajo de lo permitido.

Terminando

In [463]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6296 entries, 0 to 6295
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case_Number             6296 non-null   object 
 1   Date                    6296 non-null   object 
 2   Year                    6296 non-null   int32  
 3   Type                    6296 non-null   object 
 4   Country                 6296 non-null   object 
 5   Area                    6296 non-null   object 
 6   Location                6296 non-null   object 
 7   Activity                6296 non-null   object 
 8   Name                    6296 non-null   object 
 9   Sex                     6296 non-null   object 
 10  Age                     6296 non-null   int32  
 11  Injury                  6296 non-null   object 
 12  Fatal_(Y/N)             6296 non-null   object 
 13  Time                    6296 non-null   object 
 14  Species                 6296 non-null   objec

In [464]:
dim_original, data.shape

((25723, 24), (6296, 24))

In [465]:
data = data.reset_index(drop=True) #Reseteamos una ultima vez. Y terminamos por asignar a Case Number el indice para homogenizar.

In [466]:
data = data.set_index('Case_Number').reset_index()
data['Case_Number'] = data.index

In [468]:
data.to_csv('../Sharknado/attacks_clean.csv')